In [1]:
from __future__ import annotations

import glob
import os
import time

from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy, MlpPolicy

from pettingzoo.test import api_test
from lib.briscola_env.briscola_env import BriscolaEnv

In [2]:
env = BriscolaEnv()

api_test(env, num_cycles=1_000_000)

Starting API test


/home/tim/src/briscola-py/.venv/lib/python3.11/site-packages/pettingzoo/test/api_test.py:586: UserWarning: This environment does not have possible_agents defined.
This is not a required part 'of the API as environments with procedurally
generated agents cannot always have this property defined. However, this is
very uncommon and these features should be included whenever possible as all
standard learning code requires these properties. Also not that if you do not
have possible_agents it should also not be possible for you to expose the possible_agents
list and observation_spaces, action_spaces dictionaries.
  warnings.warn(missing_attr_warning.format(name="possible_agents"))


AttributeError: 'BriscolaEnv' object has no attribute 'terminations'